<br>
<br>

# 0. Install & import libraries 

In [1]:
from IPython.display import clear_output 

!pip install transformers==4.8.2
!pip install sentencepiece==0.1.96
!pip install tensorflow_addons

clear_output()

import sklearn
import tensorflow
import transformers
import tensorflow_addons

print(sklearn.__version__) # 1.0.2
print(tensorflow.__version__) # 2.8.0
print(transformers.__version__) # 4.8.2
print(tensorflow_addons.__version__) # 0.16.1

import sentencepiece

import pandas as pd
import numpy as np

import os
import re
import pickle 
import dill # for saving a function as a file(.pkl)
import logging # for changing the tf's logging level
import urllib.request
from tqdm import tqdm

from sklearn import model_selection
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow_addons as tfa # for using Rectified-Adam optimizer (instead of Adam optimizer) 
from tensorflow.keras import layers, initializers, losses, optimizers, metrics, callbacks 

import transformers
from transformers import TFBertModel # BertTokenizer 제외

import sentencepiece as spm

1.0.2
2.8.0
4.8.2
0.17.0


In [2]:
# Random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

# Transformers logging level 변경 (WARNING -> ERROR) @ https://huggingface.co/transformers/main_classes/logging.html
transformers.logging.set_verbosity(transformers.logging.ERROR)

# Tensorflow logging level 변경 
tf.get_logger().setLevel(logging.ERROR)

In [3]:
#KoBERT 불러오기
!git clone https://github.com/monologg/KoBERT-Transformers.git
!mv KoBERT-Transformers/kobert_transformers/tokenization_kobert.py /content
clear_output() # clear the output

#토크나이저 생성
from tokenization_kobert import KoBertTokenizer 

tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

# Import Google Drive 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
os.getcwd()

'/content'

In [6]:
# 경로만 바꾸면 됨!
path = '/content/drive/MyDrive/likelion/finalproj_seunguk'
data_path = 'colab_data/temp_data/'
checkpoint_path = 'colab_data/temp_data/saved_models/' 

os.chdir(path)

# 1. Load the Data

In [7]:
df_allsongs = pd.read_csv('df_allsongs.csv', lineterminator='\n')

In [8]:
df_allsongs.head(3)

,Unnamed: 0,Title,Artist,Date,Genre,Lyric,Lyric_ver2,preprocess_Lyric_ver2,preprocess_Lyric_ver3
0,0,나의 X에게,경서,2022.04.24,발라드,우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 밤...,"['우리 다시 만날래', '예쁘게 빛나던', '모든 추억들이', '너무 그리워', ..."
1,1,취중고백,김민석 (멜로망스),2021.12.19,발라드,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...,"['뭐하고 있었니 늦었지만', '잠시 나올래', '너의 집 골목에 있는', '놀이터..."
2,2,사랑은 늘 도망가,임영웅,2021.10.11,"발라드, 국내드라마",눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,"['눈물이 난다 이 길을 걸으면', '그 사람 손길이 자꾸 생각이 난다', '붙잡지..."


Load saved Model


In [9]:
# 1) Load the Model-builder (function)
with open(data_path + 'model_BERTfunction_v1.pkl', 'rb') as f:
    create_model = dill.loads(pickle.load(f)) # use dill to pickle a python function

# 2) Load the Bert-tokenizer 
with open(data_path + 'tokenizer-bert.pkl', 'rb') as f:
    tokenizer = pickle.load(f) 


# 3) Create the model & load the Model-weights (from checkpoint file)
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver) # Obsolete : tf.distribute.experimental.TPUStrategy()

with strategy.scope(): 
    model = create_model(max_length=300) 

model.load_weights(filepath=checkpoint_path + 'best_bert_weights.h5')

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


# Classifier as a function

In [10]:
def predict_sentiment(sentence, tokenizer, model):
    
    SEQ_LEN = 300 

    # Tokenizing / Tokens to sequence numbers / Padding
    encoded_dict = tokenizer.encode_plus(text=re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence),
                                         padding='max_length', 
                                         truncation = True,
                                         max_length=SEQ_LEN) 
    
    token_ids = np.array(encoded_dict['input_ids']).reshape(1, -1) 
    token_masks = np.array(encoded_dict['attention_mask']).reshape(1, -1)
    token_segments = np.array(encoded_dict['token_type_ids']).reshape(1, -1)
    
    new_inputs = (token_ids, token_masks, token_segments)

    # Prediction
    prediction = model.predict(new_inputs) 
    predicted_probability = np.round(np.max(prediction) * 100, 2) 
    
    predicted_class = ['분노혐오','놀람공포','슬픔','행복'][np.argmax(prediction, axis=1)[0]] # 수정하면 4개 클래스 다 나올듯! 

    print("{}% 확률로 {} 텍스트입니다.".format(predicted_probability, predicted_class))
    
    return predicted_class, predicted_probability # 태깅용 

In [11]:
new_sentence = df_allsongs.preprocess_Lyric_ver2.iloc[1]
new_sentence

'뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들 만나서 오랜만에 술을 좀 했는데 자꾸만 니 얼굴 떠올라 무작정 달려왔어 이 맘 모르겠니 요즘 난 미친 사람처럼 너만 생각해 대책없이 네가 점점 좋아져 아냐 안 취했어 진짜야 널 정말 사랑해 눈물이 날만큼 원하고 있어 정말로 몰랐니 가끔 전화해 장난치듯 주말엔 뭐할거냐며 너의 관심 끌던 나를 그리고 한번씩 누나 주려 샀는데 너 그냥 준다고 생색 낸 선물도 너 때문에 산거야 이 맘 모르겠니 요즘 난 미친 사람처럼 너만 생각해 대책없이 네가 점점 좋아져 아냐 안 취했어 진짜야 널 정말 사랑해 진심이야 믿어줘 갑자기 이런 말 놀랐다면 미안해 부담이 되는게 당연해 이해해 널 하지만 내 고백도 이해해 주겠니 oh 지금 당장 대답하진마 나와 일주일만 사귀어줄래 후회없이 잘 해주고 싶은데 그 후에도 니가 싫다면 나 그때 포기할게 귀찮게 안할게 혼자 아플게 진심이야 너를 사랑하고 있어 '

In [ ]:
predict_sentiment(new_sentence, tokenizer, model)

INFO:absl:TPU has inputs with dynamic shapes: [<tf.Tensor 'Const:0' shape=() dtype=int32>, <tf.Tensor 'cond/Identity:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_8:0' shape=(None, 300) dtype=int64>, <tf.Tensor 'cond/Identity_16:0' shape=(None, 300) dtype=int64>]


In [ ]:
for row in tqdm(df_allsongs.iterrows()):
  # #장르 
  # print(row[1][1])
  # #아티스트 
  # print(row[1][2])
  # #타이틀 
  # print(row[1][3])
  # #가사 
  print(row[1][-1])
  # #발매일 
  # print(row[1][5])
  # print(row[1][-1])
  #print(row)
  break
  #predict_sentiment(row[1][-1], tokenizer, model)

In [15]:
# Emotion = []
# Probability = []

# for row in tqdm(df_allsongs.iterrows()):
#   predicted_class, predicted_probability = predict_sentiment(row[1][-1], tokenizer, model)
#   #break
#   Emotion.append(predicted_class)
#   Probability.append(predicted_probability)

1it [00:00,  1.34it/s]

44.4% 확률로 행복 텍스트입니다.


2it [00:01,  1.36it/s]

61.14% 확률로 행복 텍스트입니다.


3it [00:02,  1.41it/s]

...

73.96% 확률로 행복 텍스트입니다.


3608it [43:06,  1.40it/s]

36.28% 확률로 행복 텍스트입니다.


3609it [43:07,  1.39it/s]

47.54% 확률로 행복 텍스트입니다.


In [16]:
# df_allsongs['Emotion'] = Emotion
# df_allsongs['Proba'] = Probability

In [17]:
# df_allsongs['Emotion'].value_counts()

행복      2346
분노혐오     917
슬픔       288
놀람공포      58
Name: Emotion, dtype: int64

In [18]:
# df_allsongs

,Unnamed: 0,Title,Artist,Date,Genre,Lyric,Lyric_ver2,preprocess_Lyric_ver2,Emotion,Proba
0,0,나의 X에게,경서,2022.04.24,발라드,우리 다시 만날래예쁘게 빛나던모든 추억들이너무 그리워너의 품에 안길래이 밤이 새도록...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",우리 다시 만날래 예쁘게 빛나던 모든 추억들이 너무 그리워 너의 품에 안길래 이 밤...,행복,44.40
1,1,취중고백,김민석 (멜로망스),2021.12.19,발라드,뭐하고 있었니 늦었지만잠시 나올래너의 집 골목에 있는놀이터에 앉아 있어친구들 만나서...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...,행복,61.14
2,2,사랑은 늘 도망가,임영웅,2021.10.11,"발라드, 국내드라마",눈물이 난다 이 길을 걸으면그 사람 손길이 자꾸 생각이 난다붙잡지 못하고 가슴만 떨...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",눈물이 난다 이 길을 걸으면 그 사람 손길이 자꾸 생각이 난다 붙잡지 못하고 가슴만...,행복,77.44
3,3,듣고 싶을까,MSG워너비(M.O.M),2022.02.26,발라드,우리 함께 듣던 그 노랠 듣고 싶을까듣고 나서 잠시 날 생각은 할까아주 혹시라도 넌...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",우리 함께 듣던 그 노랠 듣고 싶을까 듣고 나서 잠시 날 생각은 할까 아주 혹시라도...,행복,65.78
4,4,다정히 내 이름을 부르면,경서예지,2021.05.19,발라드,끝없이 별빛이 내리던 밤기분 좋은 바람이두 빰을 스치고새벽 바다 한곳을 보는아름다운...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",끝없이 별빛이 내리던 밤 기분 좋은 바람이 두 빰을 스치고 새벽 바다 한곳을 보는 ...,행복,57.01
...,...,...,...,...,...,...,...,...,...,...
3604,3604,애수,진시몬,2004.10.01,성인가요/트로트,아직도 모르겠어 난 정말 꿈이라 생각해야 하는지 너 떠난 그 길목에서 널 기다리는데...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",아직도 모르겠어 난 정말 꿈이라 생각해야 하는지 너 떠난 그 길목에서 널 ...,행복,74.71
3605,3605,단장의 미아리 고개,송가인,2020.12.26,성인가요/트로트,미아리 눈물고개임이 넘던 이별 고개화약 연기 앞을 가려눈 못 뜨고 헤매일 때당신은 ...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",미아리 눈물고개 임이 넘던 이별 고개 화약 연기 앞을 가려 눈 못 뜨고 헤매일 때 ...,행복,49.13
3606,3606,어머님 사랑합니다,송가인,2021.05.27,성인가요/트로트,꽃다운 열아홉에연지 찍고 시집와서손발이 터지도록고생하신 어머님백날을 하루같이자식 걱...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",꽃다운 열아홉에 연지 찍고 시집와서 손발이 터지도록 고생하신 어머님 백날을 하루같이...,행복,73.96
3607,3607,영동 부르스,송가인,2021.06.13,성인가요/트로트,헤어지기 싫어서 애태우던 그 날 밤피눈물에 얼룩진 그대의 모습어차피 떠나야 할인연이...,"<div class=""lyric"" id=""d_video_summary""><!-- h...",헤어지기 싫어서 애태우던 그 날 밤 피눈물에 얼룩진 그대의 모습 어차피 떠나야 할 ...,행복,36.28


In [19]:
# df_allsongs.to_csv('songtagged_1.csv')